# 1 Cargamos librerias

In [1]:
# Importaciones
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import time
import joblib

from loguru import logger

# Importar módulos del paquete product_development
from product_development.config import (
    TARGET, FEATURES, CATEGORICAL_VARS, RAW_DATA_DIR, MODELS_DIR,
    FEATURE_PIPELINE_FILE, PIPELINE_FILE, TRAIN_TEST_SPLIT_RATIO, RANDOM_STATE
)
from product_development.dataset import load_raw_data, prepare_dataset, temporal_train_test_split
from product_development.features import load_feature_pipeline, transform_features
from product_development.modeling.train import (
    get_model_configurations, train_and_evaluate_models,
    create_full_pipeline, train_final_model, save_pipeline
)

2025-11-29 10:00:20.596 | INFO     | product_development.config:<module>:17 - PROJ_ROOT path is: C:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development


Configuración de modelos, logging y rutas

In [2]:
# 2. Configuración de modelos usando el módulo train
# Modo: "fast" (rápido, 5 modelos) o "full" (completo, 15 modelos)
TRAINING_MODE = "fast"  # Cambiar a "full" para evaluación completa

models_configurations = get_model_configurations(mode=TRAINING_MODE)

print(f"Modo de entrenamiento: {TRAINING_MODE}")
print(f"Total de modelos a evaluar: {len(models_configurations)}")
print("\nModelos configurados:")
for name in models_configurations.keys():
    print(f"  - {name}")

Modo de entrenamiento: fast
Total de modelos a evaluar: 4

Modelos configurados:
  - LinearRegression
  - RandomForest
  - GradientBoosting
  - XGBoost


# 3 Entrenamiento y seleccion de modelos


In [3]:
# 3. Cargar y preparar datos usando los módulos del paquete
raw_data = load_raw_data(RAW_DATA_DIR / "train.csv")
dataset = prepare_dataset(raw_data)

print(f"Total registros: {len(dataset)}")

# Split temporal 80/20
X_train, X_val, y_train, y_val = temporal_train_test_split(dataset, TRAIN_TEST_SPLIT_RATIO)

print(f"Train: {X_train.shape}, Val: {X_val.shape}")

# Cargar pipeline de preprocesamiento
preproc_pipeline = load_feature_pipeline(FEATURE_PIPELINE_FILE)

# Ajustar pipeline con datos de entrenamiento
preproc_pipeline.fit(X_train, y_train)

# Transformar datos
X_train_proc = transform_features(preproc_pipeline, X_train)
X_val_proc = transform_features(preproc_pipeline, X_val)

print(f"Shape X_train_proc: {X_train_proc.shape}")
print(f"Shape X_val_proc: {X_val_proc.shape}")

2025-11-29 10:00:21.587 | INFO     | product_development.dataset:load_raw_data:49 - Cargando datos crudos desde C:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\data\raw\train.csv
2025-11-29 10:00:21.895 | INFO     | product_development.dataset:load_raw_data:51 - Cargados 913000 registros
2025-11-29 10:00:21.895 | INFO     | product_development.dataset:add_temporal_features:69 - Agregando características temporales
2025-11-29 10:00:22.033 | INFO     | product_development.dataset:convert_categorical_types:92 - Convirtiendo columnas categóricas a tipo object
Total registros: 913000
2025-11-29 10:00:22.082 | INFO     | product_development.dataset:temporal_train_test_split:140 - Dividiendo datos con 80% para entrenamiento
2025-11-29 10:00:21.895 | INFO     | product_development.dataset:load_raw_data:51 - Cargados 913000 registros
2025-11-29 10:00:21.895 | INFO     | product_development.dataset:add_temporal_features:69 - Agregando características temporales
2025

c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)
c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)


2025-11-29 10:00:23.290 | INFO     | product_development.features:transform_features:126 - Transformando 730400 registros


c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)
c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)


2025-11-29 10:00:24.097 | INFO     | product_development.features:transform_features:126 - Transformando 182600 registros
Shape X_train_proc: (730400, 5)
Shape X_val_proc: (182600, 5)


c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)
c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)


In [4]:
# 4. Entrenamiento y evaluación de modelos
start = time.time()

results, best_model_name, best_model = train_and_evaluate_models(
    X_train_proc, y_train,
    X_val_proc, y_val,
    models_configurations
)

elapsed_time = round(time.time() - start, 2)

print(f"\nTiempo total de entrenamiento: {elapsed_time} segundos")
print(f"\nMejor modelo: {best_model_name} con RMSE = {results[best_model_name]}")

2025-11-29 10:00:24.303 | INFO     | product_development.modeling.train:train_and_evaluate_models:157 - Iniciando entrenamiento y evaluación de modelos


Entrenando modelos:   0%|          | 0/4 [00:00<?, ?it/s]

2025-11-29 10:00:24.305 | INFO     | product_development.modeling.train:train_and_evaluate_models:160 - Entrenando modelo: LinearRegression
2025-11-29 10:00:24.381 | INFO     | product_development.modeling.train:train_and_evaluate_models:170 - Modelo LinearRegression RMSE: 30.05
2025-11-29 10:00:24.381 | INFO     | product_development.modeling.train:train_and_evaluate_models:160 - Entrenando modelo: RandomForest


Entrenando modelos:  50%|█████     | 2/4 [00:03<00:03,  1.98s/it]

2025-11-29 10:00:28.259 | INFO     | product_development.modeling.train:train_and_evaluate_models:170 - Modelo RandomForest RMSE: 27.48
2025-11-29 10:00:28.259 | INFO     | product_development.modeling.train:train_and_evaluate_models:160 - Entrenando modelo: GradientBoosting


Entrenando modelos:  75%|███████▌  | 3/4 [00:21<00:08,  8.67s/it]

2025-11-29 10:00:46.291 | INFO     | product_development.modeling.train:train_and_evaluate_models:170 - Modelo GradientBoosting RMSE: 27.13
2025-11-29 10:00:46.291 | INFO     | product_development.modeling.train:train_and_evaluate_models:160 - Entrenando modelo: XGBoost


Entrenando modelos: 100%|██████████| 4/4 [00:22<00:00,  5.75s/it]

2025-11-29 10:00:47.288 | INFO     | product_development.modeling.train:train_and_evaluate_models:170 - Modelo XGBoost RMSE: 26.92
2025-11-29 10:00:47.293 | INFO     | product_development.modeling.train:train_and_evaluate_models:181 - Mejor modelo: XGBoost con RMSE = 26.92

Tiempo total de entrenamiento: 22.99 segundos

Mejor modelo: XGBoost con RMSE = 26.92


# 4. Importamos Pipeline de preproc y Agregamos Modelo Ganador


In [5]:
# 5. Crear pipeline completo con el mejor modelo
full_pipeline = create_full_pipeline(preproc_pipeline, best_model)

print("Pipeline completo creado:")
for step_name, step in full_pipeline.steps:
    print(f"  - {step_name}: {type(step).__name__}")

2025-11-29 10:00:47.305 | INFO     | product_development.modeling.train:create_full_pipeline:205 - Creando pipeline completo con modelo
Pipeline completo creado:
  - cat_missing_imputation: SimpleCategoricalImputer
  - num_median_imputation: MeanMedianImputer
  - cat_freq_encoder: CountFrequencyEncoder
  - dayofweek_mapper: Mapper
  - feature_scaler: MinMaxScaler
  - regressor: XGBRegressor


# 5. Re entrenamos pipelines con el Modelo Ganador


In [6]:
# 6. Re-entrenar con el dataset completo
X_full = dataset[FEATURES].copy()
y_full = dataset[TARGET].copy()

# Convertir tipos categóricos
for col in CATEGORICAL_VARS:
    X_full[col] = X_full[col].astype("O")

full_pipeline = train_final_model(full_pipeline, X_full, y_full)

print("Pipeline final entrenado correctamente.")

2025-11-29 10:00:47.467 | INFO     | product_development.modeling.train:train_final_model:237 - Entrenando modelo final con dataset completo


c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)
c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_development\transformers.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[var] = X[var].fillna(self.fill_value).astype(str)
c:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\notebooks\..\product_d

2025-11-29 10:00:49.679 | INFO     | product_development.modeling.train:train_final_model:239 - Entrenamiento del modelo final completado
Pipeline final entrenado correctamente.


In [7]:
# 7. Guardar pipeline completo
save_pipeline(full_pipeline, PIPELINE_FILE)

print(f"Pipeline completo guardado en: {PIPELINE_FILE}")

2025-11-29 10:00:49.698 | INFO     | product_development.modeling.train:save_pipeline:259 - Pipeline guardado en C:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\models\sales_pipeline.pkl
Pipeline completo guardado en: C:\Users\fjgon\OneDrive - Universidad Galileo\Trimestre 8\product_development\models\sales_pipeline.pkl
